In [1]:
import pandas as pd
import networkx as nx
import os.path as osp

import torch
import torch_geometric
from torch_geometric.data import Dataset, download_url
from torch_geometric.utils.convert import from_networkx
import numpy as np

from node2vec import Node2Vec
from gensim.models import Word2Vec

import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GAE
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.nn.models.autoencoder import ARGVA

from torch_geometric.transforms import RandomLinkSplit

In [2]:
df=pd.read_csv('./BIOGRID-ORGANISM-4.4.208.tab3/BIOGRID-ORGANISM-Homo_sapiens-4.4.208.tab3.txt', sep="\t")

c:\users\calni\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,2,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.shape

(1005589, 37)

In [4]:
df_homo=df.loc[(df['Organism ID Interactor A']==9606) & (df['Organism ID Interactor B']==9606)]

In [5]:
df_homo.shape

(936634, 37)

In [6]:
df_homo.columns

Index(['#BioGRID Interaction ID', 'Entrez Gene Interactor A',
       'Entrez Gene Interactor B', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Official Symbol Interactor A',
       'Official Symbol Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Publication Source',
       'Organism ID Interactor A', 'Organism ID Interactor B', 'Throughput',
       'Score', 'Modification', 'Qualifications', 'Tags', 'Source Database',
       'SWISS-PROT Accessions Interactor A', 'TREMBL Accessions Interactor A',
       'REFSEQ Accessions Interactor A', 'SWISS-PROT Accessions Interactor B',
       'TREMBL Accessions Interactor B', 'REFSEQ Accessions Interactor B',
       'Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
       'Ontology

In [7]:
df_homo[['Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
       'Ontology Term Types']]

,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types
0,-,-,-,-,-,-
1,-,-,-,-,-,-
2,-,-,-,-,-,-
3,-,-,-,-,-,-
4,-,-,-,-,-,-
...,...,...,...,...,...,...
1005584,-,-,-,-,-,-
1005585,-,-,-,-,-,-
1005586,-,-,-,-,-,-
1005587,-,-,-,-,-,-


In [35]:
df_homo[df_homo['Ontology Term IDs']=='BTO:0000007|BTO:0000567|DOID:3702|BTO:0000661|DOID:715']

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
115451,728984,26270,79709,117654,122826,RP3-330O12.3,PSEC0241,FBXO6,COLGALT1,FBG2|FBS2|FBX6|Fbx6b,...,-,NP_078932,BTO:0000007|BTO:0000567|DOID:3702|BTO:0000661|...,HEK-293 cell|HeLa cell|cervical adenocarcinoma...,cell line|cell line|disease|cell line|disease,-|-|-|-|-,-|-|-|-|-,-|-|-|-|-,Homo sapiens,Homo sapiens
115461,728994,26270,4001,117654,110187,RP3-330O12.3,-,FBXO6,LMNB1,FBG2|FBS2|FBX6|Fbx6b,...,B4DZT3,NP_001185486|NP_005564,BTO:0000007|BTO:0000567|DOID:3702|BTO:0000661|...,HEK-293 cell|HeLa cell|cervical adenocarcinoma...,cell line|cell line|disease|cell line|disease,-|-|-|-|-,-|-|-|-|-,-|-|-|-|-,Homo sapiens,Homo sapiens


In [30]:
set(df_homo['Ontology Term IDs'])

{'-',
 'APO:0000022',
 'APO:0000050',
 'APO:0000054',
 'APO:0000058',
 'APO:0000095',
 'APO:0000106',
 'APO:0000106|APO:0000111',
 'APO:0000106|APO:0000147',
 'APO:0000111',
 'APO:0000143',
 'APO:0000144',
 'APO:0000149',
 'APO:0000155',
 'APO:0000208|APO:0000106',
 'APO:0000209',
 'BIOGRID:0000011',
 'BIOGRID:0000011|BTO:0002181',
 'BTO:0000007',
 'BTO:0000007|BTO:0000567',
 'BTO:0000007|BTO:0000567|DOID:3702',
 'BTO:0000007|BTO:0000567|DOID:3702|BTO:0000661|DOID:715',
 'BTO:0000007|BTO:0000661|DOID:715',
 'BTO:0000007|BTO:0000661|DOID:715|BTO:0000567|DOID:3702',
 'BTO:0000007|BTO:0001025',
 'BTO:0000007|BTO:0001109',
 'BTO:0000007|DOID:3702|BTO:0000567',
 'BTO:0000007|DOID:3702|BTO:0000567|DOID:715|BTO:0000661',
 'BTO:0000007|DOID:3702|BTO:0000661|BTO:0000567|DOID:715',
 'BTO:0000007|DOID:715|BTO:0000661',
 'BTO:0000007|DOID:715|BTO:0000661|DOID:3702|BTO:0000567',
 'BTO:0000017',
 'BTO:0000017|DOID:3151',
 'BTO:0000018',
 'BTO:0000018|BTO:0000017',
 'BTO:0000018|BTO:0000567|HP:000150

In [27]:
ontology_terms=set()
s=set(df_homo['Ontology Term IDs'])
for i in s:
    if i!= '-':
        l=i.split('|')
        for j in l:
            if j not in ontology_terms:
                ontology_terms.add(j)

In [29]:
len(ontology_terms)

290

In [7]:
df_homo_prot=df_homo[['Official Symbol Interactor A','Official Symbol Interactor B']]

In [8]:
df_homo_prot.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,MAP2K4,FLNC
1,MYPN,ACTN2
2,ACVR1,FNTA
3,GATA2,PML
4,RPA2,STAT3


In [9]:
num_nodes=pd.concat([df_homo_prot.iloc[:,0],df_homo_prot.iloc[:,1]]).nunique()
num_edges_original=df_homo_prot.shape[0]

In [10]:
print('Number of proteins:',num_nodes)
print('Number of edges (edges + self-loop + multiple edges):',num_edges_original)

Number of proteins: 19779
Number of edges (edges + self-loop + multiple edges): 936634


In [11]:
#remove self loop
df_homo_prot_2=df_homo_prot.loc[df_homo_prot['Official Symbol Interactor A']!=df_homo_prot['Official Symbol Interactor B']]

In [12]:
num_edges_no_sl=df_homo_prot_2.shape[0]
print('Number of edges without self loops (edges + multiple edges):',num_edges_no_sl)

Number of edges without self loops (edges + multiple edges): 929441


In [13]:
G=nx.from_pandas_edgelist(df_homo_prot,'Official Symbol Interactor A','Official Symbol Interactor B' )
print('num edges',G.number_of_edges())
G.remove_edges_from(nx.selfloop_edges(G))
print('num edges after removing self loops ',G.number_of_edges())

num edges 702531
num edges after removing self loops  699259


In [14]:
#the dataset is already filtered (without self loops)
G_2=nx.from_pandas_edgelist(df_homo_prot_2,'Official Symbol Interactor A','Official Symbol Interactor B' )
print('num edges',G_2.number_of_edges())

num edges 699259


In [15]:
#select the largest component
connected_component_subgraphs =(G.subgraph(c) for c in nx.connected_components(G))
largest_subgraph = max(connected_component_subgraphs, key=len)
print('The largest component has:',largest_subgraph.number_of_nodes())

The largest component has: 19776


In [18]:
#save the dataframe
df_homo_prot_2.to_csv('PPI.csv',index=False)

In [2]:
# import dataset
df=pd.read_csv('PPI.csv')

In [3]:
df.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,MAP2K4,FLNC
1,MYPN,ACTN2
2,ACVR1,FNTA
3,GATA2,PML
4,RPA2,STAT3


In [5]:
#create the graph
G=nx.from_pandas_edgelist(df,'Official Symbol Interactor A','Official Symbol Interactor B' )
#transform the graph in pytorch geometric dataset
#pyg_graph = from_networkx(G)
#print(pyg_graph)

In [6]:
#compute embeggin of the nodes
#node2vec = Node2Vec(G, dimensions=20, walk_length=16, num_walks=100,workers=1)

Computing transition probabilities:   0%|          | 0/19776 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|█████████████████████████████████████████████████████| 100/100 [10:28<00:00,  6.28s/it]


In [8]:
#model = node2vec.fit(window=10, min_count=1, batch_words=4) 
#save the model
#model.save('EMBEDDING_MODEL_FILENAME')
#save the embeddings
#model.wv.save_word2vec_format('EMBEDDING_FILENAME')

In [3]:
#load the model
model=Word2Vec.load('EMBEDDING_MODEL_FILENAME')

In [ ]:
#This object essentially contains the mapping between words and embeddings. 
#model.wv

In [57]:
x=torch.tensor(model.wv.get_normed_vectors())
x.shape

In [59]:
model.wv.get_vector('PWAR5')

array([ 0.7429858 ,  0.11721963,  0.03082864, -0.3415949 ,  0.5241447 ,
       -0.32755858, -0.20141901,  0.74712366,  0.34744626, -0.3101339 ,
       -0.5255988 , -0.61098903,  0.59213305, -0.17646754,  0.46805826,
       -0.8434537 ,  0.595276  ,  0.11821996,  0.13182667, -0.65747094],
      dtype=float32)

In [64]:
model.wv.get_index('PWAR5')

19259

In [66]:
model.wv.get_vector(19259)

array([ 0.7429858 ,  0.11721963,  0.03082864, -0.3415949 ,  0.5241447 ,
       -0.32755858, -0.20141901,  0.74712366,  0.34744626, -0.3101339 ,
       -0.5255988 , -0.61098903,  0.59213305, -0.17646754,  0.46805826,
       -0.8434537 ,  0.595276  ,  0.11821996,  0.13182667, -0.65747094],
      dtype=float32)

In [67]:
G.nodes['PWAR5']['x']=model.wv.get_vector('PWAR5')

In [69]:
G.nodes['PWAR5']['x']

array([ 0.7429858 ,  0.11721963,  0.03082864, -0.3415949 ,  0.5241447 ,
       -0.32755858, -0.20141901,  0.74712366,  0.34744626, -0.3101339 ,
       -0.5255988 , -0.61098903,  0.59213305, -0.17646754,  0.46805826,
       -0.8434537 ,  0.595276  ,  0.11821996,  0.13182667, -0.65747094],
      dtype=float32)

In [6]:
for n in G.nodes():
    G.nodes[n]['x']=np.array(model.wv.get_vector(n))

In [7]:
G.nodes['MAP2K4']['x']

array([-0.5213403 , -0.13549931, -0.32977045,  0.69912714,  0.4599589 ,
       -0.28745085, -0.08199365,  0.29456007, -0.21221317, -1.0883316 ,
        0.16984665, -0.5307443 ,  0.5459234 , -0.25166273,  0.4377023 ,
        0.27254134, -0.2743203 , -0.65850896, -0.07639949, -0.22601451],
      dtype=float32)

In [10]:
pyg_graph = from_networkx(G)

c:\users\calni\appdata\local\programs\python\python39\lib\site-packages\torch_geometric\utils\convert.py:178: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  data[key] = torch.tensor(value)


In [11]:
pyg_graph

Data(x=[19776, 20], edge_index=[2, 1398518])

In [11]:
type(pyg_graph)

torch_geometric.data.data.Data

In [80]:
pyg_graph.x

tensor([[-0.5213, -0.1355, -0.3298,  ..., -0.6585, -0.0764, -0.2260],
        [ 0.1210,  0.5980, -0.1422,  ..., -0.3935, -0.0862, -0.6485],
        [-0.5982,  0.7246,  0.0936,  ...,  0.0249, -0.1191, -0.3935],
        ...,
        [-0.1068,  0.7975, -0.0974,  ..., -0.6317,  0.2355, -0.5943],
        [ 0.5525,  0.0896,  0.0274,  ..., -1.0622,  1.0990, -1.5603],
        [ 0.0339,  0.7396, -0.1002,  ..., -0.9080, -0.4202,  0.1122]])

In [81]:
pyg_graph.num_node_features

20

In [82]:
pyg_graph.has_isolated_nodes()

False

In [83]:
pyg_graph.has_self_loops()

False

In [14]:
pyg_graph_3 = RandomLinkSplit(pyg_graph)

In [16]:
pyg_graph_3
#pos edges = edges present in the graph
#neg edges = edges not in the graph


RandomLinkSplit(num_val=Data(x=[19776, 20], val_pos_edge_index=[2, 34962], test_pos_edge_index=[2, 69925], train_pos_edge_index=[2, 1188744], train_neg_adj_mask=[19776, 19776], val_neg_edge_index=[2, 34962], test_neg_edge_index=[2, 69925]), num_test=0.2)

In [17]:
pyg_graph_3.num_val


Data(x=[19776, 20], val_pos_edge_index=[2, 34962], test_pos_edge_index=[2, 69925], train_pos_edge_index=[2, 1188744], train_neg_adj_mask=[19776, 19776], val_neg_edge_index=[2, 34962], test_neg_edge_index=[2, 69925])

formulates the link prediction problem as a binary classification problem as follows:

Treat the edges in the graph as positive examples.

Sample a number of non-existent edges (i.e. node pairs with no edges between them) as negative examples.

Divide the positive examples and negative examples into a training set and a test set.

Evaluate the model with any binary classification metric such as Area Under Curve (AUC).

In [101]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        #two convolutions
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        print('x.shape_enc_pre_conv1',x.shape)
        x = self.conv1(x, edge_index).relu()
        print('x.shape_enc_ppost_conv1',x.shape)
        x=self.conv2(x, edge_index)
        print('x.shape_enc_ppost_conv2',x.shape)
        return x#self.conv2(x, edge_index)

In [102]:
class GCNDecoder(torch.nn.Module):
    def __init__(self, latent_dim):
        super(GCNDecoder, self).__init__()
        #two convolutions
        self.conv1 = GCNConv(latent_dim*2,latent_dim, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(latent_dim, 1, cached=True) # cached only for transductive learning

    def forward(self, z, edge_index, sigmoid=True):
        z=torch.cat((z[edge_index[0]],z[edge_index[1]]),dim=1)
        print('z shape dopo cat',z.shape)
        z=self.conv1(z, edge_index).relu()
        print('z shape dopo conv 1 dec',z.shape)
        out=self.conv2(z, edge_index).sigmoid()
        print('out shape dopo conv 2 dec',out.shape)
        
        return out

In [103]:
def train():
    model.train()
    optimizer.zero_grad()
    #il modello ha sia l encoder ( definito da noi) che il decoder(l'inner product in questo caso)
    z = model.encode(x, train_pos_edge_index)
    #print('z',z.shape)
    #Given latent variables z, computes the binary cross entropy loss for positive edges
    loss = model.recon_loss(z, train_pos_edge_index) #reconstruction loss è un metodo del modello
    #gia presente nella classe GAE
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
        #Given latent variables z, positive edges pos_edge_index and negative edges neg_edge_index,
        #computes area under the ROC curve (AUC) and average precision (AP) scores.
        #print(z.shape)
        ##print(pos_edge_index.shape)
        #print(neg_edge_index.shape)
    return model.test(z, pos_edge_index, neg_edge_index)

In [104]:
# parameters
out_channels = 10   #embedding 
num_features = pyg_graph_3.num_val.num_features  # in_channel= number of features
epochs = 100


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
#dobbiamo passare solo l encoder , il decoder by default è l inner product
model = GAE(GCNEncoder(num_features, out_channels))

model = model.to(device)
x = pyg_graph_3.num_val.x.to(device)
train_pos_edge_index = pyg_graph_3.num_val.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [109]:
pyg_graph_3.num_val.train_pos_edge_index.shape

torch.Size([2, 1188744])

In [110]:
pyg_graph_3.num_val.test_neg_edge_index.shape

torch.Size([2, 69925])

In [111]:
1188744+69925

1258669

In [105]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(pyg_graph_3.num_val.test_pos_edge_index, pyg_graph_3.num_val.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])
x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])
Epoch: 001, AUC: 0.8966, AP: 0.8948
x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])
x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])
Epoch: 002, AUC: 0.8986, AP: 0.8965
x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])


KeyboardInterrupt: 

In [68]:
writer = SummaryWriter('runs/GAE1_experiment_'+'10d_100_epochs')

In [69]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(pyg_graph_3.num_val.test_pos_edge_index, pyg_graph_3.num_val.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.8754, AP: 0.8708
Epoch: 002, AUC: 0.8770, AP: 0.8733
Epoch: 003, AUC: 0.8798, AP: 0.8763
Epoch: 004, AUC: 0.8834, AP: 0.8803
Epoch: 005, AUC: 0.8864, AP: 0.8835
Epoch: 006, AUC: 0.8894, AP: 0.8867
Epoch: 007, AUC: 0.8928, AP: 0.8903
Epoch: 008, AUC: 0.8963, AP: 0.8939
Epoch: 009, AUC: 0.8995, AP: 0.8971
Epoch: 010, AUC: 0.9019, AP: 0.8995
Epoch: 011, AUC: 0.9035, AP: 0.9011
Epoch: 012, AUC: 0.9047, AP: 0.9022
Epoch: 013, AUC: 0.9056, AP: 0.9030
Epoch: 014, AUC: 0.9067, AP: 0.9041
Epoch: 015, AUC: 0.9082, AP: 0.9055
Epoch: 016, AUC: 0.9101, AP: 0.9073
Epoch: 017, AUC: 0.9122, AP: 0.9095
Epoch: 018, AUC: 0.9139, AP: 0.9115
Epoch: 019, AUC: 0.9143, AP: 0.9126
Epoch: 020, AUC: 0.9137, AP: 0.9130
Epoch: 021, AUC: 0.9137, AP: 0.9133
Epoch: 022, AUC: 0.9142, AP: 0.9138
Epoch: 023, AUC: 0.9143, AP: 0.9140
Epoch: 024, AUC: 0.9128, AP: 0.9132
Epoch: 025, AUC: 0.9087, AP: 0.9108
Epoch: 026, AUC: 0.9024, AP: 0.9067
Epoch: 027, AUC: 0.8978, AP: 0.9036
Epoch: 028, AUC: 0.8977, AP:

In [106]:
# parameters
latent_dim = 10   #embedding 
num_features = pyg_graph_3.num_val.num_features  # in_channel= number of features
epochs = 100


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
#dobbiamo passare solo l encoder , il decoder by default è l inner product
model = GAE(GCNEncoder(num_features, latent_dim),GCNDecoder(latent_dim))

model = model.to(device)
x = pyg_graph_3.num_val.x.to(device)
train_pos_edge_index = pyg_graph_3.num_val.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [107]:
model

GAE(
  (encoder): GCNEncoder(
    (conv1): GCNConv(20, 20)
    (conv2): GCNConv(20, 10)
  )
  (decoder): GCNDecoder(
    (conv1): GCNConv(20, 10)
    (conv2): GCNConv(10, 1)
  )
)

In [108]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(pyg_graph_3.num_val.test_pos_edge_index, pyg_graph_3.num_val.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])
z shape dopo cat torch.Size([1188744, 20])
z shape dopo conv 1 dec torch.Size([1188744, 10])
out shape dopo conv 2 dec torch.Size([1188744, 1])
z shape dopo cat torch.Size([1208520, 20])
z shape dopo conv 1 dec torch.Size([1208520, 10])
out shape dopo conv 2 dec torch.Size([1208520, 1])
x.shape_enc_pre_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv1 torch.Size([19776, 20])
x.shape_enc_ppost_conv2 torch.Size([19776, 10])
z shape dopo cat torch.Size([69925, 20])


IndexError: index out of range in self

In [70]:
from torch_geometric.nn import VGAE

In [71]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    #GCNconv restituisce una matrice (nodi,embedding) , un vettore per ogni nodo
    #quindi VGAE deve fare una sorta di pooling per avere una mu e una log sigma 
    #per ogni dimension cioè un vettore (1,embedding)

In [77]:
out_channels = 20   #embedding 
num_features = pyg_graph_3.num_val.num_features
epochs = 100


model = VGAE(VariationalGCNEncoder(num_features, out_channels)) 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = pyg_graph_3.num_val.x.to(device)
train_pos_edge_index = pyg_graph_3.num_val.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [78]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / pyg_graph_3.num_val.num_nodes) * model.kl_loss() 
    #se non viene passato nessun parametro in kl_loss allota vengono prese 
    # the last computations of mu e logsigma 
    # 
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [79]:
writer = SummaryWriter('runs/VGAE_experiment_'+'20d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(pyg_graph_3.num_val.test_pos_edge_index, pyg_graph_3.num_val.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.8806, AP: 0.8706
Epoch: 002, AUC: 0.8796, AP: 0.8702
Epoch: 003, AUC: 0.8815, AP: 0.8731
Epoch: 004, AUC: 0.8849, AP: 0.8777
Epoch: 005, AUC: 0.8886, AP: 0.8823
Epoch: 006, AUC: 0.8913, AP: 0.8856
Epoch: 007, AUC: 0.8937, AP: 0.8883
Epoch: 008, AUC: 0.8962, AP: 0.8910
Epoch: 009, AUC: 0.8992, AP: 0.8942
Epoch: 010, AUC: 0.9028, AP: 0.8979
Epoch: 011, AUC: 0.9063, AP: 0.9015
Epoch: 012, AUC: 0.9079, AP: 0.9034
Epoch: 013, AUC: 0.9067, AP: 0.9022
Epoch: 014, AUC: 0.9040, AP: 0.8996
Epoch: 015, AUC: 0.9022, AP: 0.8980
Epoch: 016, AUC: 0.9018, AP: 0.8978
Epoch: 017, AUC: 0.9021, AP: 0.8986
Epoch: 018, AUC: 0.9027, AP: 0.8999
Epoch: 019, AUC: 0.9030, AP: 0.9006
Epoch: 020, AUC: 0.9029, AP: 0.9007
Epoch: 021, AUC: 0.9026, AP: 0.9007
Epoch: 022, AUC: 0.9022, AP: 0.9007
Epoch: 023, AUC: 0.9018, AP: 0.9008
Epoch: 024, AUC: 0.9017, AP: 0.9013
Epoch: 025, AUC: 0.9022, AP: 0.9020
Epoch: 026, AUC: 0.9027, AP: 0.9029
Epoch: 027, AUC: 0.9027, AP: 0.9034
Epoch: 028, AUC: 0.9016, AP:

## ARGVAE

In [18]:
class VEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [19]:
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Discriminator, self).__init__()
        self.lin1 = torch.nn.Linear(in_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.lin3 = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x

In [36]:
def train():
    model.train()
    encoder_optimizer.zero_grad()
    
    z = model.encode(pyg_graph_3.num_val.x, pyg_graph_3.num_val.train_pos_edge_index)

    for i in range(5):
        idx = range(pyg_graph_3.num_val.num_nodes)  
        discriminator.train()
        discriminator_optimizer.zero_grad()
        discriminator_loss = model.discriminator_loss(z[idx]) # Comment
        discriminator_loss.backward(retain_graph=True)
        discriminator_optimizer.step()
 
    loss = 0
    loss = loss + model.reg_loss(z)  # Comment
    
    loss = loss + model.recon_loss(z, pyg_graph_3.num_val.train_pos_edge_index)
    loss = loss + (1 / pyg_graph_3.num_val.num_nodes) * model.kl_loss()
    loss.backward()

    encoder_optimizer.step()
    
    return loss

In [29]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [52]:
latent_size = 20
encoder = VEncoder(pyg_graph_3.num_val.num_features, out_channels=latent_size)

discriminator = Discriminator(in_channels=latent_size, hidden_channels=64, 
                              out_channels=1) # Comment

In [53]:
model = ARGVA(encoder, discriminator)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = pyg_graph_3.num_val.x.to(device)
train_pos_edge_index = pyg_graph_3.num_val.train_pos_edge_index.to(device)

In [54]:
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.005)

In [55]:
epochs=100

In [56]:
writer = SummaryWriter('runs/ARGVA_experiment_'+'20d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(pyg_graph_3.num_val.test_pos_edge_index, pyg_graph_3.num_val.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.9046, AP: 0.9009
Epoch: 002, AUC: 0.9043, AP: 0.9005
Epoch: 003, AUC: 0.9043, AP: 0.9006
Epoch: 004, AUC: 0.9048, AP: 0.9012
Epoch: 005, AUC: 0.9058, AP: 0.9023
Epoch: 006, AUC: 0.9070, AP: 0.9037
Epoch: 007, AUC: 0.9085, AP: 0.9053
Epoch: 008, AUC: 0.9103, AP: 0.9072
Epoch: 009, AUC: 0.9123, AP: 0.9093
Epoch: 010, AUC: 0.9145, AP: 0.9116
Epoch: 011, AUC: 0.9166, AP: 0.9138
Epoch: 012, AUC: 0.9181, AP: 0.9155
Epoch: 013, AUC: 0.9185, AP: 0.9160
Epoch: 014, AUC: 0.9174, AP: 0.9148
Epoch: 015, AUC: 0.9149, AP: 0.9119
Epoch: 016, AUC: 0.9110, AP: 0.9076
Epoch: 017, AUC: 0.9069, AP: 0.9030
Epoch: 018, AUC: 0.9041, AP: 0.9000
Epoch: 019, AUC: 0.9046, AP: 0.9004
Epoch: 020, AUC: 0.9077, AP: 0.9037
Epoch: 021, AUC: 0.9115, AP: 0.9078
Epoch: 022, AUC: 0.9145, AP: 0.9112
Epoch: 023, AUC: 0.9164, AP: 0.9134
Epoch: 024, AUC: 0.9176, AP: 0.9147
Epoch: 025, AUC: 0.9182, AP: 0.9154
Epoch: 026, AUC: 0.9185, AP: 0.9157
Epoch: 027, AUC: 0.9186, AP: 0.9158
Epoch: 028, AUC: 0.9184, AP: